# Worksheet 08

Name: Seunghwan Hyun 
UID: U68035800

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [6]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.728410536389913, 5.240661234299919, 4.535804263448757, 4.4816759632183425, 6.341776772541695, 4.991508386458398, 4.488737516558339, 6.931109880755049, 4.93773406317554, 4.492510555298867]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [5]:
mean = 8
stdev = 1
c2 = np.random.normal(mean, stdev, 10).tolist()
print(c2)

[7.711907791674739, 7.836471672336735, 9.21015720873368, 8.902471946936904, 8.223568080419279, 8.95757948440642, 7.683701291221868, 7.709658821052411, 7.862613110445745, 10.154657353030172]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [7]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[10.154657353030172, 7.862613110445745, 4.492510555298867, 4.93773406317554, 7.709658821052411, 6.931109880755049, 7.683701291221868, 4.488737516558339, 4.991508386458398, 6.341776772541695]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- number of distributions: 2
- p(head) = p(tail) = 0.5
- c1 ~ N(5, 1)
- c2 ~ N(8, 1)
- number of c1 samples and number of c2 samples in data

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [11]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.492510555298867, 4.93773406317554, 4.488737516558339, 4.991508386458398, 6.341776772541695]
[10.154657353030172, 7.862613110445745, 7.709658821052411, 6.931109880755049, 7.683701291221868]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 5.050453458806568,  mean_2 = 8.068348091301049
var_1 = 0.4621042335303689,  var_2 = 1.192987023770555


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [15]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))


point =  10.154657353030172
probability of observing that point if it came from cluster 0 =  2.77467532199211e-27
probability of observing that point if it came from cluster 1 =  0.07247095710867461
point =  7.862613110445745
probability of observing that point if it came from cluster 0 =  7.840608615085516e-09
probability of observing that point if it came from cluster 1 =  0.32947034402796865
point =  4.492510555298867
probability of observing that point if it came from cluster 0 =  0.4164967809609165
probability of observing that point if it came from cluster 1 =  0.0037441993209332055
point =  4.93773406317554
probability of observing that point if it came from cluster 0 =  0.838011241517947
probability of observing that point if it came from cluster 1 =  0.010688765063167951
point =  7.709658821052411
probability of observing that point if it came from cluster 0 =  5.563577054346872e-08
probability of observing that point if it came from cluster 1 =  0.3196276748361721
point =  6.

- point =  10.154657353030172
- probability of observing that point if it came from cluster 0 =  2.77467532199211e-27
- probability of observing that point if it came from cluster 1 =  0.07247095710867461
- point =  7.862613110445745
- probability of observing that point if it came from cluster 0 =  7.840608615085516e-09
- probability of observing that point if it came from cluster 1 =  0.32947034402796865
- point =  4.492510555298867
- probability of observing that point if it came from cluster 0 =  0.4164967809609165
- probability of observing that point if it came from cluster 1 =  0.0037441993209332055
- point =  4.93773406317554
- probability of observing that point if it came from cluster 0 =  0.838011241517947
- probability of observing that point if it came from cluster 1 =  0.010688765063167951
- point =  7.709658821052411
- probability of observing that point if it came from cluster 0 =  5.563577054346872e-08
- probability of observing that point if it came from cluster 1 =  0.3196276748361721
- point =  6.931109880755049
- probability of observing that point if it came from cluster 0 =  0.0002185546996141704
- probability of observing that point if it came from cluster 1 =  0.2122988722061751
- point =  7.683701291221868
- probability of observing that point if it came from cluster 0 =  7.674561903378731e-08
- probability of observing that point if it came from cluster 1 =  0.3174683381543249
- point =  4.488737516558339
- probability of observing that point if it came from cluster 0 =  0.4123972650803
- probability of observing that point if it came from cluster 1 =  0.003708854347230088
- point =  4.991508386458398
- probability of observing that point if it came from cluster 0 =  0.8563216193506817
- probability of observing that point if it came from cluster 1 =  0.012018693370283095
- point =  6.341776772541695
- probability of observing that point if it came from cluster 0 =  0.017397551822094418
- probability of observing that point if it came from cluster 1 =  0.11733873305095219
- 10.154657353030172
- Probability of coming from C_1 = 3.82867210906476e-26
- Probability of coming from C_2 = 1.0
- 7.862613110445745
- Probability of coming from C_1 = 2.3797615083171623e-08
- Probability of coming from C_2 = 0.9999999762023849
- 4.492510555298867
- Probability of coming from C_1 = 0.9910903517348022
- Probability of coming from C_2 = 0.0089096482651978
- 4.93773406317554
- Probability of coming from C_1 = 0.9874057205369582
- Probability of coming from C_2 = 0.01259427946304177
- 7.709658821052411
- Probability of coming from C_1 = 1.740642792830283e-07
- Probability of coming from C_2 = 0.9999998259357207
- 6.931109880755049
- Probability of coming from C_1 = 0.0010284083653575265
- Probability of coming from C_2 = 0.9989715916346424
- 7.683701291221868
- Probability of coming from C_1 = 2.4174253384537517e-07
- Probability of coming from C_2 = 0.9999997582574662
- 4.488737516558339
- Probability of coming from C_1 = 0.9910867584636038
- Probability of coming from C_2 = 0.008913241536396175
- 4.991508386458398
- Probability of coming from C_1 = 0.9861590056407468
- Probability of coming from C_2 = 0.01384099435925327
- 6.341776772541695
- Probability of coming from C_1 = 0.12912298894456692
- Probability of coming from C_2 = 0.870877011055433

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [17]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([(x - mean[0])**2 * prob for x, prob in zip(data, prob_c0_x)]) / sum(prob_c0_x), sum([(x - mean[1])**2 * prob for x, prob in zip(data, prob_c1_x)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.4085893673290464,  P(C_2) = 0.5914106326709536
mean_1 = 4.778680533584128,  mean_2 = 7.789651495380194
var_1 = 0.135681471837727,  var_2 = 1.4512712791104805


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [18]:
prob_c0_x = [] 
prob_c1_x = [] 
prob_x = [] 

k = 2

for p in data:
    print(f"point = {p}")
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
   
        prob_c[j]

    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point = 10.154657353030172
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.07286167826591154
point = 7.862613110445745
probability of observing that point if it came from cluster 0 =  1.9339384026514886e-112
probability of observing that point if it came from cluster 1 =  0.2745444223073197
point = 4.492510555298867
probability of observing that point if it came from cluster 0 =  0.3179981387192021
probability of observing that point if it came from cluster 1 =  0.020813786753167453
point = 4.93773406317554
probability of observing that point if it came from cluster 0 =  1.4790702923942882
probability of observing that point if it came from cluster 1 =  0.03986644527908938
point = 7.709658821052411
probability of observing that point if it came from cluster 0 =  1.3749917216934424e-101
probability of observing that point if it came from cluster 1 =  0.2744743393515317
point = 6.931109880755049
proba

- point = 10.154657353030172
- probability of observing that point if it came from cluster 0 =  0.0
- probability of observing that point if it came from cluster 1 =  0.07286167826591154
- point = 7.862613110445745
- probability of observing that point if it came from cluster 0 =  1.9339384026514886e-112
- probability of observing that point if it came from cluster 1 =  0.2745444223073197
- point = 4.492510555298867
- probability of observing that point if it came from cluster 0 =  0.3179981387192021
- probability of observing that point if it came from cluster 1 =  0.020813786753167453
- point = 4.93773406317554
- probability of observing that point if it came from cluster 0 =  1.4790702923942882
- probability of observing that point if it came from cluster 1 =  0.03986644527908938
- point = 7.709658821052411
- probability of observing that point if it came from cluster 0 =  1.3749917216934424e-101
- probability of observing that point if it came from cluster 1 =  0.2744743393515317
- point = 6.931109880755049
- probability of observing that point if it came from cluster 0 =  6.619072186341099e-55
- probability of observing that point if it came from cluster 1 =  0.2307636373087332
- point = 7.683701291221868
- probability of observing that point if it came from cluster 0 =  8.417154730036432e-100
- probability of observing that point if it came from cluster 1 =  0.2741600217689726
- point = 4.488737516558339
- probability of observing that point if it came from cluster 0 =  0.2997677609409728
- probability of observing that point if it came from cluster 1 =  0.020691142159605918
- point = 4.991508386458398
- probability of observing that point if it came from cluster 0 =  0.8592271712931329
- probability of observing that point if it came from cluster 1 =  0.04284815110849138
- point = 6.341776772541695
- probability of observing that point if it came from cluster 0 =  4.456642277937848e-29
- probability of observing that point if it came from cluster 1 =  0.1671203967315245
- 10.154657353030172
- Probability of coming from C_1 = 0.0
- Probability of coming from C_2 = 1.0

- 7.862613110445745
- Probability of coming from C_1 = 4.86662569808472e-112
- Probability of coming from C_2 = 1.0

- 4.492510555298867
- Probability of coming from C_1 = 0.9134597754331011
- Probability of coming from C_2 = 0.08654022456689886

- 4.93773406317554
- Probability of coming from C_1 = 0.9624509152038315
- Probability of coming from C_2 = 0.037549084796168565

- 7.709658821052411
- Probability of coming from C_1 = 3.460957511126487e-101
- Probability of coming from C_2 = 1.0

- 6.931109880755049
- Probability of coming from C_1 = 1.9816532978468184e-54
- Probability of coming from C_2 = 1.0

- 7.683701291221868
- Probability of coming from C_1 = 2.1210901026499996e-99
- Probability of coming from C_2 = 1.0

- 4.488737516558339
- Probability of coming from C_1 = 0.909166693187319
- Probability of coming from C_2 = 0.09083330681268109

- 4.991508386458398
- Probability of coming from C_1 = 0.9326778734513682
- Probability of coming from C_2 = 0.06732212654863172

- 6.341776772541695
- Probability of coming from C_1 = 1.842367443376596e-28
- Probability of coming from C_2 = 1.0

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [22]:
assignments = []

for i in zip(prob_c0_x, prob_c1_x):
    p1, p2 = i
    
    if p1 > p2:
        assignments.append(0)
    else:
        assignments.append(1)
    
print(assignments)

[1, 1, 0, 0, 1, 1, 1, 0, 0, 1]
